In [24]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import linear_model, metrics

In [5]:
data = pd.read_csv("./chevron.csv")
# Clean data:
# get rid of zeros
# make sure that empty cells is filled in with 0
# drop non states (US, etc.)
# lots of 0's
# get rid of random features that are duplicates
# normalize the features
# adjust the weights
# LASSO, some other method

In [6]:
alaska_df = data.loc[data["StateCode"] == "AK"]
num_rows = 4
num_cols = 1
y = pd.DataFrame(index=range(num_rows), columns=["TotalAmountofAssistance"])
investment_vals = []
for i in range(num_rows):
    investment_vals.append(data["TotalAmountofAssistance"][1519 * i])
for j in range(num_rows):
    y.at[j, "TotalAmountofAssistance"] = investment_vals[j]
# y.at[0]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][0]
# print(y)
# y.at[1]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][1519]
# y.at[2]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][3038]
# y.at[3]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][4557]
# y.at[4]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][6076]
# for investment_idx in range(5):
#     print(investment_idx)
#     print(data["TotalAmountofAssistance"][3038])
#     row_idx = 1519 * investment_idx
#     y.at[investment_idx]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][row_idx]
print(y)


  TotalAmountofAssistance
0               3345612.0
1               2247734.0
2              13509662.0
3              18236626.0


In [11]:
print(alaska_df)

      Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0              0  BDFDB        AK  2015     21.0  Alaska            35.027804   
1              1  BDPRP        AK  2015      4.0  Alaska            35.027804   
2              2  BFFDB        AK  2015     21.0  Alaska            35.027804   
3              3  BFPRP        AK  2015      4.0  Alaska            35.027804   
4              4  CLPRB        AK  2015  17747.0  Alaska            35.027804   
...          ...    ...       ...   ...      ...     ...                  ...   
6100        6100  WDPRB        AK  2019   6418.0  Alaska            34.259440   
6101        6101  WDTCB        AK  2019   6418.0  Alaska            34.259440   
6102        6102  WSTCB        AK  2019    386.0  Alaska            34.259440   
6103        6103  WWPRB        AK  2019   6803.0  Alaska            34.259440   
6104        6104  WYTCB        AK  2019   1270.0  Alaska            34.259440   

      TotalNumberofInvestme

In [20]:
numRows = 4
numCols = 29
x = pd.DataFrame(index=range(numRows), columns=range(numCols))
x.columns = alaska_df["MSN"][0:numCols]
x_test = pd.DataFrame(index=range(4, 5), columns=range(numCols))
x_test.columns = alaska_df["MSN"][0:numCols]
# print(x.columns)
avgs = [[0] * numCols] * numRows
for row_idx in range(int(alaska_df.shape[0])):
    yr_idx = int(row_idx / numCols)
    # col_name = alaska_df["MSN"][row_idx % 29]
    # print(col_name)
    if row_idx < int(alaska_df.shape[0] * 4 / 5):
        x.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]
    else:
        x_test.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]


print(x)
print(x_test)


MSN   BDFDB   BDPRP   BFFDB   BFPRP     CLPRB   CLPRK   CLPRP  COPRK   EMFDB  \
0      21.0     4.0    21.0     4.0   17747.0  15.073  1177.0  5.717     0.0   
1    1906.0   351.0  1906.0   351.0  247632.0   25.68  9643.0  5.722     0.0   
2    6760.0  1244.0  6760.0  1244.0     832.0  19.472    43.0  5.723     0.0   
3       0.0     0.0  6758.0  1184.0  140759.0  21.489  6550.0  5.706  6758.0   

MSN   ENPRP  ...     REPRB    SOTCB      TEPRB      TETCB   WDEXB     WDPRB  \
0       0.0  ...   23807.0      3.0  1429942.0   614242.0     0.0    6865.0   
1       0.0  ...  230734.0    431.0  1118407.0  1922727.0     0.0  162166.0   
2       0.0  ...  116692.0    442.0   989005.0  1057701.0  5455.0   77206.0   
3    1184.0  ...  156071.0  70779.0   622052.0  1542208.0     0.0    9307.0   

MSN     WDTCB   WSTCB     WWPRB   WYTCB  
0      6865.0   622.0    7488.0  1488.0  
1    162166.0  1606.0  163772.0     0.0  
2     71751.0  4359.0   81565.0     0.0  
3      9307.0   489.0    9796.0  48

In [15]:
reg = linear_model.LinearRegression()
reg.fit(x, y)

LinearRegression()

In [17]:
print(reg.intercept_)
print(reg.coef_)

[44282875.20623823]
[[ 2.86635690e-03  5.18891269e-04  1.96331924e-01  3.44140129e-02
  -3.58501597e+00 -9.77300849e-05 -1.12485482e-01 -4.12308474e-07
   1.93465567e-01  3.38951216e-02  1.14967346e-02  4.84687070e-03
   2.16073245e+00 -1.77221009e+00 -1.55839293e-06 -1.27511987e+00
  -2.12660022e+00 -8.53748841e+00 -1.49318464e+00 -1.63995420e+00
   2.01714211e+00 -1.76612975e+01 -9.03932418e+00  4.89683097e-02
  -3.99698098e+00 -4.04594929e+00 -3.02426042e-05 -3.99701857e+00
   1.27246738e-01]]


In [23]:
y_pred = reg.predict(x_test)
print(y_pred)

[[-78000470.01822823]]


In [25]:
mse = metrics.mean_squared_error(y, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [4, 1]